In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# Polygon Meshes
todo

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P28.jpg" style="width: 200px; display: inline;">


## Elements of a Mesh

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P12.jpg" style="width: 600px; display: inline;">


## Mesh Objects in Decod.es

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/3.00.D68 Mesh Large.jpg" style="width: 800px; display: inline;">

In [ ]:
"""
Mesh Initialization
Meshes are constructed in much the same way as other HasPts objects, relying on 
arguments passed through to the superclass constructor for the defining of basic 
members, and only then initializing a private collection to store faces.
"""
class Mesh(HasPts):
    def __init__(self, vertices=None, faces=None, basis=None):
         #HasPts constructor handles initalization of verts and basis
        super(Mesh,self).__init__(vertices,basis)
        self._faces = [] if (faces is None) else faces

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P13.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P14.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P15.jpg" style="width: 200px; display: inline;">


In [ ]:
"""
Face Management
The Decod.es Mesh offers minimal methods for managing faces. In the first method
below, a single face that relates three or four Points is added to the private 
collection _faces. The second method below defines a property faces that 
provides access to this private collection.
"""
def add_face(self,a,b,c,d=None):
    if is not None : self._faces.append([a,b,c,d])
    else: self._faces.append([a,b,c])
    
@property
def faces(self):
    return self._faces

In [ ]:
"""
Querying Face Properties
Given the minimal structure of a Decod.es Mesh, a range of geometric properties 
may be calculated for any mesh face identified by index. The first function 
below simply returns the points of the desired face. Using this information, 
the second calculates its centroid. Finally, the third calculates the normal 
vector of the desired face, accounting for four-sided faces where necessary.
"""
def face_pts(self,idx):
    return [self.pts[i] for i in self.faces[idx]]
    
def face_centroid(self,idx):
    return Point.centroid(self.face_pts(idx))
    
def face_normal(self,idx):
    pts = self.face_pts(idx)
    va = Vec(pts[0],pts[1]).cross(Vec(pts[0],pts[2])).normalized()
    if len(verts) == 3 : return va
    
    vb = Vec(pts[2],pts[3]).cross(Vec(pts[2],pts[1])).normalized()
    return Vec.bisector(va,vb).normalized()

### Mesh Usage

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P21.jpg" style="width: 200px; display: inline;">


In [ ]:
"""
Mesh Quad Corner-to-Center Subdivision
"""
def quadsub_corner_to_ctr(msh, fac_idx):
    # append a Point at the face center, and find index
    msh.append( msh.face_centroid(fac_idx) )
    ctr_idx = len(msh)-1
    # find indices of corners of existing face
    cnr_idxs = msh._faces[fac_idx]
    
    # add four new faces
    msh.add_face(cnr_idxs[0],cnr_idxs[1],ctr_idx)
    msh.add_face(cnr_idxs[1],cnr_idxs[2],ctr_idx)
    msh.add_face(cnr_idxs[2],cnr_idxs[3],ctr_idx)
    msh.add_face(cnr_idxs[3],cnr_idxs[0],ctr_idx)
    
    # remove the existing face
    del msh._faces[fac_idx]